In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
from zipfile import ZipFile
import urllib
from datetime import datetime
import string
import pygsheets
import requests
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import Bio.Entrez as Entrez
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.primers as primers
import paulssonlab.cloning.ncbi as ncbi

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Backbone primers for Eaton library Gibson

In [ ]:
parts = reg[("LIB", "parts")]
olib = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]

In [ ]:
olib.next_id()

In [ ]:
olib["oLIB177"]

In [ ]:
olib["oLIB178"]

In [ ]:
olib["oLIB176"]

In [ ]:
parts["UNS10"]["Sequence"]

In [ ]:
template = plib_maps["pLIB112"]

## Check that replace_primer_overhang works

In [ ]:
primers.replace_primer_overhang(
    template, olib["oLIB177"]["Sequence"], parts["UNS10"]["Sequence"]
)

In [ ]:
primers.replace_primer_overhang(
    template,
    olib["oLIB177"]["Sequence"],
    sequence.reverse_complement(parts["UNS10"]["Sequence"]),
)

In [ ]:
primers.replace_primer_overhang(
    template, olib["oLIB178"]["Sequence"], parts["UNS1"]["Sequence"]
)

## Generate new primers

In [ ]:
primer_r = str(
    primers.replace_primer_overhang(
        template,
        olib["oLIB177"]["Sequence"],
        parts["Eaton_library_prefix"]["Sequence"].upper(),
    )
)

In [ ]:
primer_f = str(
    primers.replace_primer_overhang(
        template,
        olib["oLIB178"]["Sequence"],
        parts["Eaton_library_suffix"]["Sequence"].upper(),
    )
)

In [ ]:
date = datetime.now().strftime("%-m/%-d/%Y")
description = "Primers for amplifying pLIB112 storage vector with appropriate 20bp Eaton flanks for Gibsoning in a library."
base = {
    "Author": "Jacob Quinn Shenker",
    "Date": date,
    "Order date": date,
    "Vendor": "IDT",
    "Type": "Primer",
    "Description": description,
}
olib[olib.next_id()] = {
    "Sequence": primer_r,
    "Name": "Eaton_flanks_storage_gibson_r",
    **base,
}
olib[olib.next_id()] = {
    "Sequence": primer_f,
    "Name": "Eaton_flanks_storage_gibson_f",
    **base,
}

In [ ]:
olib.save()